In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import pandas as pd
import numpy as np
realizer_path = os.path.join(os.environ['SLREALIZERDIR'], 'slrealizer')
sys.path.insert(0, realizer_path)

# How to interface with SLRealizer

Author: Ji Won Park (jiwoncpark)

This notebook demonstrates how to create an SLRealizer instance and use its methods to emulate the LSST DRP Source and Object catalogs ("source and object tables"). We will use the OM10Realizer subclass to realize lensed quasar systems from OM10 (Oguri and Marshall 2010) and the SDSS subclass to realize galaxies from SDSS DR14 (Abolfathi et al. 2018).

To use SLRealizer, we need two types of catalogs: one defining the static properties of objects such as size and flux ("object catalog") and another defining the observation conditions such as atmospheric seeing and the cadence schedule ("observation catalog"). The subclasses of SLRealizer are named after the object catalog; OM10Realizer realizes objects in the OM10 mock quasar catalog and SDSSRealizer realizes objects in SDSS data release products.

We will use the same observation catalog in our realization of lenses and non-lenses. Called `minion_1016`, this is the current official Reference Simulated Survey for the LSST. Let's begin by defining the path to our observation catalog.

In [ ]:
# This is the directory where all our data files will be stored.
data_dir = os.path.join(os.environ['SLREALIZERDIR'], 'data')
# Observation catalog filename
observation_fname = os.path.join(data_dir, 'twinkles_observation_history.csv')
# Observation catalog dataframe
observation_df = pd.read_csv(observation_fname).query("(filter != 'y')").reset_index(drop=True)

Note I have queried out entries with the y-band; this is simply because the SDSS does not have a y-band!

## 1. Realizing lensed quasar systems with OM10Realizer

We first define the paths for the output source and object tables.

In [ ]:
output_lens_source_path = os.path.join(data_dir, 'lens_source_table.csv')
output_lens_object_path = os.path.join(data_dir, 'lens_object_table.csv')

We then use OM10's native interface instead of Pandas to read in the OM10 catalog. 

In [ ]:
from om10 import DB

# OM10 catalog filename
om10_fname = os.path.join(data_dir, 'qso_mock.fits')
# OM10's DB object
om10_db = DB(catalog=om10_fname)
# Selection to simulate LSST conditions
om10_db.select_random(maglim=23.3, area=1.e2, IQ=0.75)
# Giving colors using a framework called LensPop (Collett 2015)
om10_db.paint(synthetic=True)

More about the selection line: `maglim` is the magnitude of the third brightest quasar image in the lens system. Note that, when querying a catalog of blended objects, we can only select on the total magnitude of the system (all the quasar images and the lens galaxy) so we need to be inclusive with the `maglim` selection here and then apply cuts later as necessary. `IQ` is the survey image quality. `area` refers to the total survey area, but entering a number above 1.e5 sq. ft. allows us to get all the lenses that meet our `maglim` and `IQ` criteria. We advise keeping the area small, say 100.0, if you just want to see that the cell runs!

Now we're ready to instantiate OM10Realizer.

In [ ]:
from realize_om10 import OM10Realizer

# OM10 instance
om10realizer = OM10Realizer(observation=observation_df,
                        catalog=om10_db, 
                        debug=False, add_moment_noise=True, add_flux_noise=True)

The `debug` mode simply returns more objects and is more verbose. The two noise-related keywords `add_moment_noise` and `add_flux_noise` ask whether you want to add random noise to the output catalog, and are passed into the constructor because they are a source of randomness (which you might not want when debugging).

Finally, we can make our source and object tables. When making the source table, we can optionally choose to include the intrinsic quasar time variability. Note that object tables are generated by time-averaging the source table for each filter, so the `make_object_table` requires a source table as input.

In [ ]:
import warnings; warnings.simplefilter('ignore')

# Make source table
om10realizer.make_source_table_vectorized(output_source_path=output_lens_source_path,
                                          include_time_variability=True)
# Make object table from the source table just made
om10realizer.make_object_table(include_std=False,
                               source_table_path=output_lens_source_path,
                               object_table_path=output_lens_object_path)

## 2. Realizing non-lenses with SDSSRealizer

The steps for realizing non-lenses are similar! Let's define the output catalog paths.

In [ ]:
output_nonlens_source_path = os.path.join(data_dir, 'nonlens_source_table.csv')
output_nonlens_object_path = os.path.join(data_dir, 'nonlens_object_table.csv')

We read in the object catalog using Pandas. You might want to sample a small number of objects (say, 20) if you just want to see that the cells run.

In [ ]:
# SDSS catalog filename
sdss_fname = os.path.join(data_dir, 'sdss_processed.csv')
# SDSS catalog dataframe
sdss_df = pd.read_csv(sdss_fname).sample(20, random_state=123).reset_index(drop=True)

As above, we instantiate SDSSRealizer and make the source and object tables. Note that, since SDSS objects are not lenses, there is no option to add intrinsic variability.

In [ ]:
from realize_sdss import SDSSRealizer

sdssrealizer = SDSSRealizer(observation=observation_df, 
                            catalog=sdss_df, 
                            debug=False, add_moment_noise=True, add_flux_noise=True)

sdssrealizer.make_source_table_vectorized(save_path=output_nonlens_source_path)
sdssrealizer.make_object_table(include_std=True,
                               source_table_path=output_nonlens_source_path,
                               object_table_path=output_nonlens_object_path)